# Router Query Engine

In [ ]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.4/702.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [ ]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    ListIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
)

import openai
openai.api_key = 'YOUR_OPENAI_API_KEY'

In [ ]:
# load documents
documents = SimpleDirectoryReader("data").load_data()

# initialize service context (set chunk size)
service_context = ServiceContext.from_defaults(chunk_size=1024)
nodes = service_context.node_parser.get_nodes_from_documents(documents)

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Define List Index and Vector Index over Same Data

In [ ]:
list_index = ListIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

# Define Query Engines and Set Metadata

In [ ]:
list_query_engine = list_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.tools.query_engine import QueryEngineTool


list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description="Useful for summarization questions related to Paul Graham eassy on What I Worked On.",
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context from Paul Graham essay on What I Worked On.",
)

# Define Router Query Engine

## PydanticSingleSelector

In [ ]:
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.selectors.pydantic_selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)


query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

In [ ]:
response = query_engine.query("What is the summary of the document?")

In [ ]:
response.response

"The document provides a personal account of the author's experiences and interests in writing, programming, art, and entrepreneurship. It discusses their early experiences in writing and programming, their interest in artificial intelligence, and their transition to art. The document also explores their time working at Interleaf, their decision to start a company called Viaweb, and the challenges and successes they faced in building the software and navigating the startup landscape. Additionally, it touches on the author's involvement with Y Combinator and their decision to retire and pursue other interests. The document provides insights into the author's journey and decision-making process throughout their career."

## LLMSingleSelector

In [ ]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

In [ ]:
response = query_engine.query("What is the summary of the document?")
response.response

"The document provides a personal account of the author's experiences and interests in writing, programming, art, and entrepreneurship. It starts with their early experiences in writing and programming, their fascination with artificial intelligence, and their decision to focus on Lisp programming. The author also discusses their interest in art, their experiences at art school, and their exploration of still life painting. The document then transitions to the author's experiences working at Interleaf, attending art school, and starting a company called Viaweb. It describes the challenges and successes of building the software and launching the company. The author later reflects on their decision to leave Viaweb, pursue other interests, and start a new company focused on web applications. The document also touches on the author's involvement in various projects, such as spam filters and creating Hacker News. It concludes with the author's experiences leading up to the creation of Y Com

In [ ]:
response = query_engine.query("What did Paul Graham do after RICS?")
response.response

'After leaving RISD, Paul Graham moved to New York and resumed his old life. He continued to paint and experimented with a new kind of still life. He also looked for an apartment to buy and eventually got the idea to build a web app for making web apps. He decided to move to Cambridge and start this new venture, recruiting a team to work on it.'

## PydanticMultiSelector

In [ ]:
from llama_index import SimpleKeywordTableIndex

keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context using keywords from Paul Graham essay on What I Worked On.",
)

In [ ]:
query_engine = RouterQueryEngine(
    selector=PydanticMultiSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
        keyword_tool,
    ],
)

In [ ]:
# This query could use either a keyword or vector query engine, so it will combine responses from both
response = query_engine.query(
    "What were noteable events and people from the authors time at Interleaf and YC?"
)

In [ ]:
response.response

"The notable events and people from the author's time at Interleaf are not mentioned in the provided context information. However, during the author's time at Y Combinator (YC), they co-founded a company called Viaweb with Robert Morris and received seed funding from Julian Weber. The author's deal with Julian became the model for Y Combinator's future deals with founders. The author also recruited Sam Altman to become the president of YC, marking a changing of the guard and a reorganization of the company. They also worked closely with Jessica Livingston, who played a significant role in YC. The context mentions the growth and success of YC, as well as the decision to transition from being a fund back to being self-funded."